# Pyspark and SQL

Useful ressources: 
* https://www.tutorialspoint.com/spark_sql/spark_sql_quick_guide.htm
* https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

Create DataFrames

In [ ]:
from pyspark import SparkContext
sc = SparkContext()
import pyspark

In [ ]:
# import pyspark class Row from module sql
from pyspark.sql import *

# Create Example Data - Departments and Employees

# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)

# Create the DepartmentWithEmployees instances from Departments and Employees
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4])
departmentWithEmployees3 = Row(department=department3, employees=[employee1, employee4])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])

print department1
print employee2
print departmentWithEmployees1.employees[0].email


In [ ]:
from pyspark import SQLContext
sqlContext = SQLContext(sc)


Create DataFrames from a list of the rows

In [ ]:
departmentsWithEmployeesSeq1 = [departmentWithEmployees1, departmentWithEmployees2]
df1 = sqlContext.createDataFrame(departmentsWithEmployeesSeq1)

df1.show()
        
departmentsWithEmployeesSeq2 = [departmentWithEmployees3, departmentWithEmployees4]
df2 = sqlContext.createDataFrame(departmentsWithEmployeesSeq2)

df2.show()


# Work with DataFrames
## Union two DataFrames

In [ ]:
unionDF = df1.unionAll(df2)
unionDF.show()


Write the unioned DataFrame to a Parquet file

In [ ]:
# Remove the file if it exists
unionDF.write.parquet("./databricks-df-example.parquet")


Read a DataFrame from the Parquet file

In [ ]:
parquetDF = sqlContext.read.parquet("./databricks-df-example.parquet")
parquetDF.show()

## Explode the employees column

In [ ]:
from pyspark.sql.functions import explode
df = unionDF.select(explode("employees").alias("e"))
explodeDF = df.selectExpr("e.firstName", "e.lastName", "e.email", "e.salary")

explodeDF.show()


## Use filter() to return the rows that match a predicate

In [ ]:
filterDF = explodeDF.filter(explodeDF.firstName == "xiangrui").sort(explodeDF.lastName)
filterDF.show()


In [ ]:
from pyspark.sql.functions import col, asc

# Use `|` instead of `or`
filterDF = explodeDF.filter((col("firstName") == "xiangrui") | (col("firstName") == "michael")).sort(asc("lastName"))
filterDF.show()


The where() clause is equivalent to filter()

In [ ]:
whereDF = explodeDF.where((col("firstName") == "xiangrui") | (col("firstName") == "michael")).sort(asc("lastName"))
whereDF.show()


Replace null values with -- using DataFrame Na function

In [ ]:
nonNullDF = explodeDF.fillna("--")
nonNullDF.show()


Retrieve only rows with missing firstName or lastName

In [ ]:
filterNonNullDF = explodeDF.filter(col("firstName").isNull() | col("lastName").isNull()).sort("email")
filterNonNullDF.show()


Example aggregations using agg() and countDistinct()

In [ ]:
from pyspark.sql.functions import countDistinct

countDistinctDF = explodeDF.select("firstName", "lastName")\
  .groupBy("firstName", "lastName")\
  .agg(countDistinct("firstName"))

countDistinctDF.show()


## Compare the DataFrame and SQL query physical plans

In [ ]:
countDistinctDF.explain()
# register the DataFrame as a temp table so that we can query it using SQL
explodeDF.registerTempTable("databricks_df_example")

# Perform the same query as the DataFrame above and return ``explain``
countDistinctDF_sql = sqlContext.sql("SELECT firstName, lastName, count(distinct firstName) as distinct_first_names FROM databricks_df_example GROUP BY firstName, lastName")

countDistinctDF_sql.explain()


Sum up all the salaries

In [ ]:
salarySumDF = explodeDF.agg({"salary" : "sum"})
salarySumDF.show()

type(explodeDF.salary)



Print the summary statistics for the salaries

In [ ]:
explodeDF.describe("salary").show()


An example using pandas and Matplotlib integration

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.clf()
pdDF = nonNullDF.toPandas()
pdDF.plot(x='firstName', y='salary', kind='bar', rot=45)
plt.show()

See also for more information: 
* https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html
* https://spark.apache.org/docs/latest/sql-programming-guide.html